In [0]:
import urllib.request 
from urllib.parse import quote 
from bs4 import BeautifulSoup
import json
import re
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Source: https://codezup.com/web-scraping-word-meaning-dictionary-python-beautifulsoup/

In [0]:
import json
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/bigFreqList.json') as json_file:
     wordsData = json.load(json_file)

In [0]:
#generate a list of words with frequency higher than the threshold
def generateWordList(threshold): 
  temp = []
  for (word, freq) in wordsData.items():
    if (freq >= threshold):
      temp.append(word)
  return temp

In [6]:
wordList = generateWordList(40)
print(wordList[-10:])
len(wordList)

['latikivn', 'imusicvn', 'ycđt', 'greatland', 'lemmevn', 'hoclamgiau', 'hpschoolvn', 'khochiasevn', 'hakiba', 'heisvn']


141836

In [0]:
#break the word list into batches of 
batches = [wordList[i:i + 10000] for i in range(0, len(wordList), 10000)] 

In [8]:
for batch in batches:
  print(len(batch))

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
1836


In [0]:
#The link format for vdict Viet-Eng is: https://vdict.com/m%E1%BB%99t,2,0,0.html

In [0]:
#Converting IRI to ASCII:
#https://stackoverflow.com/questions/4389572/how-to-fetch-a-non-ascii-url-with-python-urlopen

In [0]:
def findTranslation(word):
    print("Finding translation for " + word)
    url = "https://vdict.com/" + quote(word) + ",2,0,0.html" #resolving the IRI issue
    
    try: #make sure the link is working
        source = urllib.request.urlopen(url)
    except:
        print("Link broken for " + word)
        return "N/A"
    
    soup = BeautifulSoup(source, 'lxml')

    #make sure the word is in the dictionary
    if (soup.find("div", id = 'result-contents') is None):
        print("Not in the dictionary " + word)
        return "N/A"
    
    translations = {}

    #Getting the first POS tag
    firstPosTag = soup.find("div", class_ = 'phanloai')
    
    if (firstPosTag is None): #empty translation page
        print("Empty translation page " + word)
        return "N/A"
    
    curPos = firstPosTag.string.split(' \xa0')[0]

    #first translation is a tag away from the first POS tag
    curTag = firstPosTag.next_sibling.next_sibling

    #storing the number of translations a word has
    numTrans = 1

    #continue while we are either in a POS tag or a translation tag
    while (curTag != '\n'): 
        if (curTag.attrs["class"] == ["list1"]): #a translation
            temp = {} #temp dict to store each translation of a word

            #getting the raw translation
            if (curTag.find('b').string is not None): #empty translation cell
                temp['translation'] = re.split('[,;]', curTag.find('b').string)
                temp['POS'] = curPos

                #retrieving the examples of each of the translations
                try:
                    examples = curTag.findAll("ul", class_= "list2")
                    temp2 = {} #another dict to store each examples of a translation

                    numExamples = len(list(examples))
                    temp2['nums'] = numExamples

                    for j in range(0, numExamples):
                        temp2['context ' + str(j+1)] = examples[j].find(class_ = "example-original").string

                        #retrieving the usage of each examples
                        try: 
                            tempString = str(examples[j].find("li")) #extracting the usage
                            temp2['usage ' + str(j+1)] = str(examples[j].find("li").contents[-1])
                        except:
                            temp2['usage ' + str(j+1)] = "N/A"

                    temp['examples'] = temp2
                except:
                    temp['examples'] = "N/A"

                translations[numTrans] = temp #adding the translation to the dictionary

                numTrans += 1
            curTag = curTag.next_sibling                
        else: #a tag
            curPos = curTag.string.split(' \xa0')[0]
            curTag = curTag.next_sibling

    #storing the number of translations
    translations['nums'] = numTrans - 1
    return translations

In [0]:
url = "https://vdict.com/" + quote("có") + ",2,0,0.html" #resolving the IRI issue

try:
    source = urllib.request.urlopen(url)
except:
    print( "Link broken")
soup = BeautifulSoup(source, 'lxml')

In [13]:
soup.find("div", class_ = 'phanloai').next_sibling.next_sibling.next_sibling.next_sibling

'\n'

In [0]:
from google.colab import files
start_time = time.time()

index = 2
batch = batches[index]

#for (index, batch) in enumerate(batches):
dictionary = {}

#making the dictionary for each batch
for word in batch:
    dictionary[word] = findTranslation(word)
    time.sleep(1) #wait for 1 sec between each query

Streaming output truncated to the last 5000 lines.
Not in the dictionary giót
Finding translation for germanycarcomvn
Not in the dictionary germanycarcomvn
Finding translation for gardena
Not in the dictionary gardena
Finding translation for garan
Not in the dictionary garan
Finding translation for gt3
Not in the dictionary gt3
Finding translation for focusrite
Not in the dictionary focusrite
Finding translation for fabinho
Not in the dictionary fabinho
Finding translation for fts
Not in the dictionary fts
Finding translation for eurokera
Not in the dictionary eurokera
Finding translation for essentials
Not in the dictionary essentials
Finding translation for endresshauser
Not in the dictionary endresshauser
Finding translation for emmerson
Not in the dictionary emmerson
Finding translation for elengant
Not in the dictionary elengant
Finding translation for ecomarathon
Not in the dictionary ecomarathon
Finding translation for earphone
Not in the dictionary earphone
Finding translation 

In [0]:
dictionary = {}

In [0]:
#saving the batch
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/vdict VI-EN batch {}.json'.format(index), 'w') as outfile:
  json.dump(dictionary, outfile)

print("--- %s seconds ---" % (time.time() - start_time))

In [0]:
findTranslation("cái")